In [1]:
import numpy as np
import pandas as pd
import os
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:80% !important; }</style>")) 

# Load original ratings data

In [2]:
cwd = os.getcwd()
ratings = pd.read_csv(os.path.join(cwd, "..", "data", "ratings.csv"))
ratings.drop('timestamp', axis=1, inplace=True)

In [3]:
max_userId = ratings['userId'].max()
ryan_id = max_userId + 1
print('Ryan userid: ' + str(ryan_id))

Ryan userid: 138494


# Load and clean Ryan's rating data

In [4]:
ryan = pd.read_csv('Ryans_Movie_Ratings_complete.csv')
ryan.replace(to_replace=['', ' ', 'NaN'], value=np.nan, inplace=True)
ryan.dropna(subset=['rating'], inplace=True)

In [5]:
ryan.head()

,rating,title
6,4,Whip It (2009)
49,5,Daybreakers (2010)
64,3,Austin Powers in Goldmember (2002)
70,3,Final Destination 3 (2006)
71,3,"Fast and the Furious, The (2001)"


In [6]:
ryan['rating'].value_counts()

4    144
3    105
5     91
2     25
1      3
Name: rating, dtype: int64

#### Merge 'movieId' back into Ryan's data

In [7]:
cwd = os.getcwd()
movies = pd.read_csv(os.path.join(cwd, "..", "data", "movies.csv"))
ryan = pd.merge(ryan, movies[['movieId', 'title']], on='title')

#### Remove Aladdin because it has two ID's

In [8]:
ryan[ryan.duplicated(subset=['title'], keep=False)]

,rating,title,movieId
64,4,Aladdin (1992),588
65,4,Aladdin (1992),114240


In [9]:
ryan = ryan.query('movieId != 114240')

# Persist profile

In [11]:
ryan[['movieId', 'rating']].to_csv('ryan_profile.csv')

# Union Ryan's ratings with the original ratings data

In [10]:
ryan.drop(['title'], inplace=True, axis=1)

In [11]:
ryan['userId'] = ryan_id
ryan = ryan[['userId', 'movieId', 'rating']]

In [12]:
ratings_combined = pd.concat([ratings, ryan])

In [13]:
ratings_combined.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


# Persist combined dataset

In [14]:
cwd = os.getcwd()
ratings_combined.to_csv(os.path.join(cwd, "..", "data", "ratings_combined_ryan.csv"), index=False)